
# Generando el dataset para el proyecto de clasificación de noticias

Utilizamos los archivos csv proporcionados para crear nuestro dataset inicial (disponibles en noticias)

In [1]:
import pandas as pd
from os import listdir
from os.path import join
from tqdm import tqdm
from my_funcs import clean_and_tag
import re
from pandasql import sqldf
import numpy as np

In [2]:
pd.read_csv('./noticias/chile_2021-01.csv').head(3)

,id,country,media_outlet,url,title,text,date,year,id_journalist
0,3816243,chile,elmostrador,https://www.elmostrador.cl/dia/2021/01/01/ley-...,Ley de Migración: diputada Núñez (RN) formula ...,La diputada y vicepresidenta de Renovación Nac...,2021-01-01,2021.0,NaN
1,3816387,chile,elmostrador,https://www.elmostrador.cl/destacado/2021/01/0...,2021 arranca con 3.588 nuevos contagios y 52 f...,"Según el reporte de hoy, el Ministerio de Salu...",2021-01-01,2021.0,NaN
2,3816512,chile,elmostrador,https://www.elmostrador.cl/dia/2021/01/01/829-...,829 personas fueron detenidas durante noche de...,Carabineros reportó que hasta las 06:00 am. de...,2021-01-01,2021.0,NaN


 Ya que estas noticias no se encuentran etiquetadas, es necesario realizar un procesamiento previo para asignarles una categoría que nos permita entrenar y probar nuestros modelos de clasificación.
 
 ### **Categorías a utilizar**: Se utilizarán las categorias propuestas en el enunciado del trabajo 
 
* 1- Mundo
* 2- Economía
* 3- Política y Conflictos
* 4- Ciencias y Tecnología,
* 5- Catástrofes y Accidentes,
* 6- Cultura y Artes, 
* 7- Deporte,
* 8- Ecología y Planeta,
* 9- Crimen, delitos y Justicia, 
* 10- Salud

### Adoptamos dos estrategias para realizar esta tarea:

1. Asignarle una categoría a partir de la información de la url.
2. Utilizar LDA para encontrar tópicos que correspondan a alguna categoría.

# 1. Etiquetado a partir de la información contenida en la url.

Debido a la gran cantidad de noticias, propusimos inicialmente utilizar esta metodología para disminuir el trabajo inicial y aprovechando la propia información que nos proveen los medios de prensa.

Todas las noticias en su URL vienen con una categoria propia del medio, aunque esta pueda diferir entre los medios (una misma noticias para un medio puede ser *nacional* pero para otro puede ser *cultura*) podemos clasificar inicialmente utilizando esta información.

In [3]:
unique_url_tags = set()
folder = './noticias'
dfs = []
for archivo in tqdm(listdir(folder)):
    df = pd.read_csv(join(folder, archivo))
    corpus = clean_and_tag(df)
    dfs.append(corpus)
    for tag in corpus.tag.unique():
        unique_url_tags.add(tag)

100%|██████████| 8/8 [00:03<00:00,  2.38it/s]


In [4]:
print(len(unique_url_tags))
list(unique_url_tags)[:10]

83


['mineria',
 'sin-editar',
 'artes',
 'la-tercera-pm',
 'turismo',
 'economia',
 'calidad-de-vida',
 'mouse',
 'noticias',
 'programas']

#### Encontramos **83** tags distintos. Revisamos esta lista y asignamos manualmente una categoría a los url_tags que pertenecieran a alguna de las 10 categorías objetivo (este trabajo manual se encuentra en el archivo ```categorias - hojav2.csv```)

In [5]:
dm = pd.read_csv('./intermediate_data/categorias - hojav2.csv')
dm.rename(columns={'Unnamed: 0':'tag'}, inplace=True)
vals = dm.iloc[:, 1:]
labels = vals[vals.max(axis=1)>0].idxmax(axis=1)
print(len(labels))
labels.head(5)

40


1       8-Ecología_y_Planeta
2          6-Cultura_y_Artes
3    4-Ciencias_y_Tecnología
4    3-Política_y_Conflictos
8    4-Ciencias_y_Tecnología
dtype: object

### Utilizamos un diccionario para manejar de mejor manera las *tag_url* con las categorías propuestas

In [6]:
cats = pd.DataFrame(labels)
cats = cats.join(dm['tag'])
cats.rename(columns={0:'cat'}, inplace=True)
cats_dict = dict(zip(cats['tag'], cats['cat']))
list(cats_dict.items())[:5]

[('animal', '8-Ecología_y_Planeta'),
 ('artes', '6-Cultura_y_Artes'),
 ('Autos', '4-Ciencias_y_Tecnología'),
 ('braga', '3-Política_y_Conflictos'),
 ('ciencia-tecnologia', '4-Ciencias_y_Tecnología')]

Se agrega la columna ```category``` al dataframe rellenando con NaN. 
Creamos una columna *content* que contiene el título de la noticias más el cuerpo de esta.
Esto se realizó debido a que el título contiene información sumamente importante la cual explica la idea principal de esta en muy pocas lineas de texto, por lo que utilizarla nos proporciona información útil para su clasificación.

In [7]:
df = pd.concat(dfs)
df.drop_duplicates(subset='url', keep='first', inplace=True)
df['category'] = df.apply(lambda row : cats_dict.get(row.tag, np.nan), axis=1)
#df['content'] = df['title'] + df['text']
df['content'] = df[['title', 'text']].agg('. '.join, axis=1)
df.head(3)

,id,country,media_outlet,url,title,text,date,tag,category,content
0,49234,chile,horas24,https://www.24horas.cl/coronavirus/tia-pikachu...,"""Tía Pikachu"" por parte empadronado: ""Estoy co...","Ha sufrido los embates del carro lanza aguas,...",2020-09-01,coronavirus,10-Salud,"""Tía Pikachu"" por parte empadronado: ""Estoy co..."
3,49291,chile,horas24,https://www.24horas.cl/nacional/rm-alcanza-sus...,RM alcanza sus mejores índices de calidad de a...,"La ministra del Medio Ambiente, Carolina Schm...",2020-09-01,nacional,NaN,RM alcanza sus mejores índices de calidad de a...
6,49335,chile,horas24,https://www.24horas.cl/nacional/paro-de-camion...,Paro de camioneros: Supermercados preocupados ...,"Esta jornada, la sexta de paralización de los...",2020-09-01,nacional,NaN,Paro de camioneros: Supermercados preocupados ...


In [8]:
df.id.max()

17798719

Almacenamos 30.000 de las noticias cuyo *url_tag* era muy general o no nos servía para asignarle una categoria. A estas noticias en una segunda etapa le aplicaremos LDA  para poder etiquetarlas y así aumentar la información de nuestro dataset inicial.

In [9]:
pd.set_option('display.max_colwidth', 120)
df = df[['id','url','title', 'content', 'category']]
unravel = df[df['category'].isna()]
(unravel.sample(n=30000, random_state=2)).to_csv('./intermediate_data/30k_sin_cat.csv', index=False)
df = df[df['category'].notna()]
final_df_link = df.drop(columns=['url'])
final_df_link.to_csv('./intermediate_data/final_df_link.csv', index=False)
final_df_link.head(3)

,id,title,content,category
0,49234,"""Tía Pikachu"" por parte empadronado: ""Estoy complicada de aquí a diciembre""","""Tía Pikachu"" por parte empadronado: ""Estoy complicada de aquí a diciembre"". Ha sufrido los embates del carro lanza...",10-Salud
7,49349,Sernac recibió más de 400 reclamos durante la primera jornada de CyberDay,"Sernac recibió más de 400 reclamos durante la primera jornada de CyberDay. Luego de la primera jornada de CyberDay,...",2-Economía
22,49598,"¿Cómo y dónde pueden abrir? Este miércoles reabren restaurantes en ""modo pandemia""","¿Cómo y dónde pueden abrir? Este miércoles reabren restaurantes en ""modo pandemia"". Este miércoles 2 de septiembre ...",10-Salud


Luego de esto, podemos ver que logramos etiquetar de buena manera una cantidad considerable de noticias, pero, existen categorías las cuales no pudimos proveerles una gran cantidad de noticias y existieron otras, particularmente la categoria *'5- Catástrofes y Accidentes'* que no fue posible, esto es debido a que los medios de prensa no tienen en sus URL una categoria para catastrofes o accidentes, siendo muy probable que se encuentren dentro de la categoría *'país'*.

Por otro lado, debido al rango de fechas utilizado y a la forma de asignar categorías según la url, es posible que nuestro dataset esté sesgado, por ejemplo, en la categoría de *Salud*. Debido a que todas las noticias son a partir de Septiembre de 2020, fechas las cuales la gran mayoría de noticias son acerca de la pandemia del Coronavirus, por lo que noticias anteriores (o futuras) que hablen de la influenza u otras enfermedades que hayan pasado a segundo plano no sea bien clasificadas por el modelo final. Esto es aún más relevante al asignar la categoria **Salud** a las noticias con **coronavirus** en la url.


In [10]:
q="""SELECT category, count(*) FROM final_df_link GROUP BY category ORDER BY count(*) DESC;"""
url_cat_result=sqldf(q)
url_cat_result

,category,count(*)
0,1-Mundo,8609
1,7-Deporte,6708
2,4-Ciencias_y_Tecnología,4225
3,2-Economía,2636
4,3-Política_y_Conflictos,1923
5,10-Salud,333
6,6-Cultura_y_Artes,185
7,8-Ecología_y_Planeta,135
8,9-Crimen_delitos_y_Justicia,130


Como es posible ver, la mitad de las categorías está considerablemente subrepresentada.

# 2. Aplicación de LDA a noticias no clasificadas

A las 30.000 noticias que guardamos antes le aplicamos LDA con 20 tópicos, creemos que es un buen número para poder apreciar una diferencia entre los grupos formados, debido a la gran cantidad de noticias a analizar.

Posteriormente se realizará un clustering más fino con LDA adicional para aquellas noticias que queden en un tópico no muy bien definido o sin un tópico equivalente a las categorías objetivo.


In [11]:
import re
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import spacy
%matplotlib inline

In [12]:
df = pd.read_csv('./intermediate_data/30k_sin_cat.csv')
df.head(3)

,id,url,title,content,category
0,6354016,https://www.latercera.com/que-pasa/noticia/vacuna-de-la-u-de-oxford-la-mas-avanzada-del-mundo-comenzara-sus-pruebas-...,"Vacuna de la U. de Oxford, la más avanzada del mundo, anuncia inicio de sus pruebas Fase 3 en Chile","Vacuna de la U. de Oxford, la más avanzada del mundo, anuncia inicio de sus pruebas Fase 3 en Chile. La Universidad ...",NaN
1,4685035,https://www.latercera.com/nacional/noticia/caso-corpesca-tribunal-condena-a-jaime-orpis-por-cohecho-y-delitos-reiter...,Caso Corpesca: Tribunal condena a Jaime Orpis por cohecho y delitos reiterados de fraude al fisco,Caso Corpesca: Tribunal condena a Jaime Orpis por cohecho y delitos reiterados de fraude al fisco. Uno de los casos ...,NaN
2,7317217,https://www.meganoticias.cl/tendencias/323633-gianluca-vacchi-edad-sharon-fonseca-blu-jerusalema-rpx04.html,Desde un lujoso auto hasta viajes por el mundo: Así enamoró Gianluca Vacchi a Sharon Fonseca,Desde un lujoso auto hasta viajes por el mundo: Así enamoró Gianluca Vacchi a Sharon Fonseca. Gianluca Vacchi (53) s...,NaN


In [13]:
def sent_to_words(sentences):
    for sentence in tqdm(sentences, desc="sentences to words"):
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [14]:
#import nltk
#nltk.download('stopwords')

#!python3 -m spacy download es_core_news_md

In [15]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
stop_words = stopwords.words('spanish')
nlp = spacy.load('es_core_news_md', disable=['parser', 'ner'])
def remove_stopwords(texts):
    desc = "removing stopwords"
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in tqdm(texts, desc=desc)]

def make_bigrams(texts, bigram_mod):
    return [bigram_mod[doc] for doc in tqdm(texts, desc="making bigrams")]

def make_trigrams(texts, bigram_mod, trigram_mod):
    return [trigram_mod[bigram_mod[doc]] for doc in tqdm(texts, desc="making trigrams")]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in tqdm(texts, desc="lemmatization"):
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [16]:
def get_ngrams(data_words):
    print('getting n-grams...')
    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return bigram, trigram, bigram_mod, trigram_mod

In [17]:
def get_lemmatized_data(df):
    data = df.content.values.tolist()
    data = [re.sub(r'\s+', ' ', sent) for sent in tqdm(data, desc="replacing whitespaces")]
    data_words = list(sent_to_words(data))
    bigram, trigram, bigram_mod, trigram_mod = get_ngrams(data_words)
    data_words_nostops = remove_stopwords(data_words)
    data_words_bigrams = make_bigrams(data_words_nostops, bigram_mod)
    data_lemmatized = lemmatization(data_words_bigrams,
                                    allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    return data_lemmatized

In [20]:
import pickle
try:
    with open ('./intermediate_data/data_lemmatized', 'rb') as fp:
        data_lemmatized = pickle.load(fp)
except FileNotFoundError:
    print('data_lemmatized not found...')
    data_lemmatized = get_lemmatized_data(df)
    with open('./intermediate_data/data_lemmatized', 'wb') as fp:
        pickle.dump(data_lemmatized, fp)

### Entrenamiento del modelo de tópico con LDA

- Entrenaremos un primer modelo de tópicos buscando un modelo con 20 tópicos. 

In [21]:
model_filename = "./models/model_lda_20_topics.mm"
num_topics = 20
#lda_model = gensim.models.ldamodel.LdaModel.load("./models/model_lda_20_topics.mm", )

In [22]:
%%time
try:
    lda_model = gensim.models.ldamodel.LdaModel.load(model_filename)
    id2word = corpora.Dictionary.load(model_filename + ".id2word")  
except FileNotFoundError:
    print("LDA model...")
    id2word = corpora.Dictionary(data_lemmatized)
    corpus = [id2word.doc2bow(text) for text in tqdm(data_lemmatized, desc='corpus')]
#     lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
#                                            id2word=id2word,
#                                            num_topics=num_topics, 
#                                            random_state=100,
#                                            update_every=1,
#                                            chunksize=100,
#                                            passes=10,
#                                            alpha='auto',
#                                            per_word_topics=True)
    
    lda_model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=num_topics,
                                           chunksize=100,
                                           passes=10,
                                           per_word_topics=True,
                                                   workers=4)
    lda_model.save(model_filename)

CPU times: user 84.1 ms, sys: 20.8 ms, total: 105 ms
Wall time: 142 ms


In [23]:
def format_topics_documents(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(tqdm(ldamodel[corpus])):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [24]:
domtop_filename = './intermediate_data/df_dominant_topic.csv'
try:
    df_dominant_topic = pd.read_csv(domtop_filename)
except FileNotFoundError:
    df_topic_sents_keywords = format_topics_documents(lda_model, corpus, data_lemmatized)
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.to_csv(domtop_filename, index=False)
finally:    
    display(df_dominant_topic.head(5))

,index,Dominant_Topic,Perc_Contribution,Topic_Keywords,0
0,0,7.0,0.9958,"vacuna, persona, dosis, salud, primero, virus, ano, paciente, medico, mas","['vacuna', 'avanzado', 'mundo', 'anunciar', 'inicio', 'prueba', 'laboratorio_astrazeneca', 'anunciar', 'vacuna', 'pr..."
1,1,0.0,0.4724,"caso, publico, derecho, abogado, tribunal, fiscal, ley, delito, recurso, hecho","['caso', 'cohecho', 'delitos_reiterado', 'fraude', 'fisco', 'caso', 'emblematico', 'financiamiento_irregular', 'poli..."
2,2,14.0,0.8776,"instagrama_post, post_on, view_this, ano, post_shared, hacer, mas, hijo, ver, vida","['lujoso', 'auto', 'mundo', 'enamorar', 'convertido', 'influencer', 'popular', 'redes_social', 'millón', 'seguidor',..."
3,3,2.0,0.5394,"persona, comuna, medida, fase, salud, semana, cuarentena, permiso, hora, autoridad","['comercio', 'preocupado', 'retroceso', 'fase', 'manifesto', 'preocupacion', 'retroceso', 'fase', 'común', 'marco', ..."
4,4,2.0,0.3311,"persona, comuna, medida, fase, salud, semana, cuarentena, permiso, hora, autoridad","['detener', 'persona', 'protestar', 'afuera', 'catedral', 'personal', 'primero', 'comisaria', 'carabineros', 'detene..."


In [25]:
df_dominant_topic.columns = ['index', 'Dominant_Topic',
                             'Perc_Contribution',
                             'Keywords', 'text_list']

In [26]:
df_topics = pd.concat([df.drop(columns=['category']), df_dominant_topic[['Dominant_Topic',
                                                                         'Keywords',
                                                                         'text_list']]], axis=1)


In [27]:
df_topics['Dominant_Topic'] = df_topics['Dominant_Topic'].astype(int)
df_topics.head(3)

,id,url,title,content,Dominant_Topic,Keywords,text_list
0,6354016,https://www.latercera.com/que-pasa/noticia/vacuna-de-la-u-de-oxford-la-mas-avanzada-del-mundo-comenzara-sus-pruebas-...,"Vacuna de la U. de Oxford, la más avanzada del mundo, anuncia inicio de sus pruebas Fase 3 en Chile","Vacuna de la U. de Oxford, la más avanzada del mundo, anuncia inicio de sus pruebas Fase 3 en Chile. La Universidad ...",7,"vacuna, persona, dosis, salud, primero, virus, ano, paciente, medico, mas","['vacuna', 'avanzado', 'mundo', 'anunciar', 'inicio', 'prueba', 'laboratorio_astrazeneca', 'anunciar', 'vacuna', 'pr..."
1,4685035,https://www.latercera.com/nacional/noticia/caso-corpesca-tribunal-condena-a-jaime-orpis-por-cohecho-y-delitos-reiter...,Caso Corpesca: Tribunal condena a Jaime Orpis por cohecho y delitos reiterados de fraude al fisco,Caso Corpesca: Tribunal condena a Jaime Orpis por cohecho y delitos reiterados de fraude al fisco. Uno de los casos ...,0,"caso, publico, derecho, abogado, tribunal, fiscal, ley, delito, recurso, hecho","['caso', 'cohecho', 'delitos_reiterado', 'fraude', 'fisco', 'caso', 'emblematico', 'financiamiento_irregular', 'poli..."
2,7317217,https://www.meganoticias.cl/tendencias/323633-gianluca-vacchi-edad-sharon-fonseca-blu-jerusalema-rpx04.html,Desde un lujoso auto hasta viajes por el mundo: Así enamoró Gianluca Vacchi a Sharon Fonseca,Desde un lujoso auto hasta viajes por el mundo: Así enamoró Gianluca Vacchi a Sharon Fonseca. Gianluca Vacchi (53) s...,14,"instagrama_post, post_on, view_this, ano, post_shared, hacer, mas, hijo, ver, vida","['lujoso', 'auto', 'mundo', 'enamorar', 'convertido', 'influencer', 'popular', 'redes_social', 'millón', 'seguidor',..."


# Análisis manual

In [28]:
from my_funcs import get_df_keywords
get_df_keywords(lda_model, num_topics)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,caso,hacer,persona,ministro,carabinero,proyecto,ano,vacuna,ano,lugar,millón,presidente,partido,violencia,instagrama_post,incendio,mas,alcalde,caso,comuna
1,publico,decir,comuna,presidente,vehiculo,diputado,colegio,persona,delito,ano,empresa,voto,candidato,persona,post_on,bombero,ano,local,nuevo,region
2,derecho,mas,medida,general,lugar,ley,social,dosis,mujer,ver,pago,elección,politico,manifestación,view_this,sector,agua,persona,cifra,antofagasta
3,abogado,ver,fase,interior,persona,parlamentario,vivo,salud,victima,zona,persona,electoral,constituyente,octubre,ano,pic_twitter,producto,mesa,total,común
4,tribunal,creer,salud,carabinero,personal,comision,programa,primero,imputado,obra,pesos,estadounidense,primario,estallido_social,post_shared,emergencia,nuevo,municipio,contagio,zona
5,fiscal,persona,semana,director,detenido,retiro,director,virus,hecho,ciudad,servicio,resultado,nuevo,sismo,hacer,vivienda,estudio,plebiscito,nivel,metro
6,ley,gente,cuarentena,jefe,hecho,fondo,profesor,ano,joven,tierra,realizar,ano,independiente,hora,mas,trabajar,animal,vecino,persona,nota_tabla_emol_tablaennoticio
7,delito,querer,permiso,hecho,arma,senador,nuevo,paciente,fiscal,espacio,dinero,internacional,proceso,protesta,hijo,lugar,explicar,comuna,aumento,estación
8,recurso,hoy,hora,cargo,policial,constitucional,mujer,medico,menor,tercero,pagar,primero,elección,grupo,ver,afectado,tambien,votacion,salud,lago
9,hecho,tener,autoridad,autoridad,encontrar,reforma,trabajo,mas,hombre,evento,contar,país,acuerdo,zona,vida,encontrar,cientifico,votar,región,puente_alto



    1- Mundo
    2- Economía
    3- Política y Conflictos
    4- Ciencias y Tecnología,
    5- Catástrofes y Accidentes,
    6- Cultura y Artes,
    7- Deporte,
    8- Ecología y Planeta,
    9- Crimen, delitos y Justicia,
    10- Salud


Analizando los 20 tópicos determinamos que cada uno de estos corresponde a las siguentes categorías propuestas.

Algunos tópicos como el 6, 13 y 19 serán procesados con LDA nuevamente para poder mejorar su clasificación debido a que en esta primera instancia no fue posible clasificarlos de buena manera.

In [29]:
url_cat_result

,category,count(*)
0,1-Mundo,8609
1,7-Deporte,6708
2,4-Ciencias_y_Tecnología,4225
3,2-Economía,2636
4,3-Política_y_Conflictos,1923
5,10-Salud,333
6,6-Cultura_y_Artes,185
7,8-Ecología_y_Planeta,135
8,9-Crimen_delitos_y_Justicia,130


In [30]:
# nos importan mas aquellas categorias menos numerosas
cats_dict = {0 : '9-Crimen_delitos_y_Justicia',
             3 : '3-Política_y_Conflictos',
             4 : '9-Crimen_delitos_y_Justicia',
             5 : '3-Política_y_Conflictos',
             7 : '10-Salud',
             8 : '9-Crimen_delitos_y_Justicia',
             10 : '2-Economía',
             11 : '3-Política_y_Conflictos',
             12 : '3-Política_y_Conflictos',
             15 : '5-Catástrofes_y_Accidentes'}

# el resto aun sin categoria
for i in range(0, num_topics):
    if i not in cats_dict.keys():
        cats_dict[i] = np.nan
        
cats_dict
df_topics['category'] = df_topics.apply(lambda row : cats_dict[row.Dominant_Topic], axis=1)
df_topics.head(10)

,id,url,title,content,Dominant_Topic,Keywords,text_list,category
0,6354016,https://www.latercera.com/que-pasa/noticia/vacuna-de-la-u-de-oxford-la-mas-avanzada-del-mundo-comenzara-sus-pruebas-...,"Vacuna de la U. de Oxford, la más avanzada del mundo, anuncia inicio de sus pruebas Fase 3 en Chile","Vacuna de la U. de Oxford, la más avanzada del mundo, anuncia inicio de sus pruebas Fase 3 en Chile. La Universidad ...",7,"vacuna, persona, dosis, salud, primero, virus, ano, paciente, medico, mas","['vacuna', 'avanzado', 'mundo', 'anunciar', 'inicio', 'prueba', 'laboratorio_astrazeneca', 'anunciar', 'vacuna', 'pr...",10-Salud
1,4685035,https://www.latercera.com/nacional/noticia/caso-corpesca-tribunal-condena-a-jaime-orpis-por-cohecho-y-delitos-reiter...,Caso Corpesca: Tribunal condena a Jaime Orpis por cohecho y delitos reiterados de fraude al fisco,Caso Corpesca: Tribunal condena a Jaime Orpis por cohecho y delitos reiterados de fraude al fisco. Uno de los casos ...,0,"caso, publico, derecho, abogado, tribunal, fiscal, ley, delito, recurso, hecho","['caso', 'cohecho', 'delitos_reiterado', 'fraude', 'fisco', 'caso', 'emblematico', 'financiamiento_irregular', 'poli...",9-Crimen_delitos_y_Justicia
2,7317217,https://www.meganoticias.cl/tendencias/323633-gianluca-vacchi-edad-sharon-fonseca-blu-jerusalema-rpx04.html,Desde un lujoso auto hasta viajes por el mundo: Así enamoró Gianluca Vacchi a Sharon Fonseca,Desde un lujoso auto hasta viajes por el mundo: Así enamoró Gianluca Vacchi a Sharon Fonseca. Gianluca Vacchi (53) s...,14,"instagrama_post, post_on, view_this, ano, post_shared, hacer, mas, hijo, ver, vida","['lujoso', 'auto', 'mundo', 'enamorar', 'convertido', 'influencer', 'popular', 'redes_social', 'millón', 'seguidor',...",NaN
3,167926,https://www.biobiochile.cl/noticias/nacional/region-de-la-araucania/2020/10/01/camara-de-comercio-preocupada-por-ret...,Cámara de Comercio preocupada por retroceso a fase 2 de Temuco y otras 10 comuna de La Araucanía,Cámara de Comercio preocupada por retroceso a fase 2 de Temuco y otras 10 comuna de La Araucanía. La Cámara de Comer...,2,"persona, comuna, medida, fase, salud, semana, cuarentena, permiso, hora, autoridad","['comercio', 'preocupado', 'retroceso', 'fase', 'manifesto', 'preocupacion', 'retroceso', 'fase', 'común', 'marco', ...",NaN
4,4738784,https://www.latercera.com/nacional/noticia/carabineros-detuvo-a-ocho-personas-en-protesta-a-las-afueras-de-la-catedr...,Carabineros detuvo a ocho personas en protesta a las afueras de la Catedral de Concepción,Carabineros detuvo a ocho personas en protesta a las afueras de la Catedral de Concepción. Personal de la Primera Co...,2,"persona, comuna, medida, fase, salud, semana, cuarentena, permiso, hora, autoridad","['detener', 'persona', 'protestar', 'afuera', 'catedral', 'personal', 'primero', 'comisaria', 'carabineros', 'detene...",NaN
5,6318450,https://www.latercera.com/que-pasa/noticia/mujeres-manifiestan-mas-rechazo-que-los-hombres-a-la-vacuna-contra-el-cov...,Mujeres manifiestan más rechazo que los hombres a la vacuna contra el Covid,Mujeres manifiestan más rechazo que los hombres a la vacuna contra el Covid. De acuerdo a lo que han informado las a...,7,"vacuna, persona, dosis, salud, primero, virus, ano, paciente, medico, mas","['mujer', 'manifestar', 'rechazo', 'hombre', 'vacuna', 'acuerdo', 'informado', 'autoridad', 'chileno', 'tendrar', 'a...",10-Salud
6,3966295,https://www.elmostrador.cl/dia/2020/10/26/minsal-detalla-protocolos-para-uso-de-piscinas-cines-y-gimnasios-durante-l...,"Minsal detalla protocolos para uso de piscinas, cines y gimnasios durante la fase 4 del plan ""Paso a Paso""","Minsal detalla protocolos para uso de piscinas, cines y gimnasios durante la fase 4 del plan ""Paso a Paso"". Durante ...",2,"persona, comuna, medida, fase, salud, semana, cuarentena, permiso, hora, autoridad","['minsal', 'detallar', 'protocolo', 'uso', 'piscina', 'cines_gimnasios', 'paso', 'balance_diario', 'lunes', 'subsecr...",NaN
7,7323819,https://www

In [31]:
df_sin_topic = df_topics[df_topics.category.isna()]

In [32]:
df_topics = df_topics[['id','url','title', 'content', 'category']]
df_topics = df_topics[df_topics['category'].notna()]
df_topics.drop(columns=['url'], inplace=True)
df_topics.head(3)

,id,title,content,category
0,6354016,"Vacuna de la U. de Oxford, la más avanzada del mundo, anuncia inicio de sus pruebas Fase 3 en Chile","Vacuna de la U. de Oxford, la más avanzada del mundo, anuncia inicio de sus pruebas Fase 3 en Chile. La Universidad ...",10-Salud
1,4685035,Caso Corpesca: Tribunal condena a Jaime Orpis por cohecho y delitos reiterados de fraude al fisco,Caso Corpesca: Tribunal condena a Jaime Orpis por cohecho y delitos reiterados de fraude al fisco. Uno de los casos ...,9-Crimen_delitos_y_Justicia
5,6318450,Mujeres manifiestan más rechazo que los hombres a la vacuna contra el Covid,Mujeres manifiestan más rechazo que los hombres a la vacuna contra el Covid. De acuerdo a lo que han informado las a...,10-Salud


In [33]:
final_df_link = pd.read_csv('./intermediate_data/final_df_link.csv')
final_df_link.head(3)

,id,title,content,category
0,49234,"""Tía Pikachu"" por parte empadronado: ""Estoy complicada de aquí a diciembre""","""Tía Pikachu"" por parte empadronado: ""Estoy complicada de aquí a diciembre"". Ha sufrido los embates del carro lanza...",10-Salud
1,49349,Sernac recibió más de 400 reclamos durante la primera jornada de CyberDay,"Sernac recibió más de 400 reclamos durante la primera jornada de CyberDay. Luego de la primera jornada de CyberDay,...",2-Economía
2,49598,"¿Cómo y dónde pueden abrir? Este miércoles reabren restaurantes en ""modo pandemia""","¿Cómo y dónde pueden abrir? Este miércoles reabren restaurantes en ""modo pandemia"". Este miércoles 2 de septiembre ...",10-Salud


In [34]:
q="""SELECT category, count(*) FROM final_df_link GROUP BY category ORDER BY count(*) DESC;"""
result=sqldf(q)
result

,category,count(*)
0,1-Mundo,8609
1,7-Deporte,6708
2,4-Ciencias_y_Tecnología,4225
3,2-Economía,2636
4,3-Política_y_Conflictos,1923
5,10-Salud,333
6,6-Cultura_y_Artes,185
7,8-Ecología_y_Planeta,135
8,9-Crimen_delitos_y_Justicia,130


In [35]:
q="""SELECT category, count(*) FROM df_topics GROUP BY category ORDER BY count(*) DESC;"""
result=sqldf(q)
result

,category,count(*)
0,9-Crimen_delitos_y_Justicia,7275
1,3-Política_y_Conflictos,5164
2,10-Salud,1610
3,2-Economía,862
4,5-Catástrofes_y_Accidentes,703


In [36]:
final_df = pd.concat([final_df_link, df_topics]).reset_index().drop(columns=['index'])

In [37]:
q="""SELECT category, count(*) FROM final_df GROUP BY category ORDER BY count(*) DESC;"""
result=sqldf(q)
result

,category,count(*)
0,1-Mundo,8609
1,9-Crimen_delitos_y_Justicia,7405
2,3-Política_y_Conflictos,7087
3,7-Deporte,6708
4,4-Ciencias_y_Tecnología,4225
5,2-Economía,3498
6,10-Salud,1943
7,5-Catástrofes_y_Accidentes,703
8,6-Cultura_y_Artes,185
9,8-Ecología_y_Planeta,135


In [38]:
final_df.to_csv('./datasets/final_df_paso_1.csv', index=False)

In [39]:
df_topics.groupby('category').size()

category
10-Salud                       1610
2-Economía                      862
3-Política_y_Conflictos        5164
5-Catástrofes_y_Accidentes      703
9-Crimen_delitos_y_Justicia    7275
dtype: int64

In [40]:
print(len(df_sin_topic))
df_sin_topic[['title', 'text_list', 'category']].head(5)

14386


,title,text_list,category
2,Desde un lujoso auto hasta viajes por el mundo: Así enamoró Gianluca Vacchi a Sharon Fonseca,"['lujoso', 'auto', 'mundo', 'enamorar', 'convertido', 'influencer', 'popular', 'redes_social', 'millón', 'seguidor',...",NaN
3,Cámara de Comercio preocupada por retroceso a fase 2 de Temuco y otras 10 comuna de La Araucanía,"['comercio', 'preocupado', 'retroceso', 'fase', 'manifesto', 'preocupacion', 'retroceso', 'fase', 'común', 'marco', ...",NaN
4,Carabineros detuvo a ocho personas en protesta a las afueras de la Catedral de Concepción,"['detener', 'persona', 'protestar', 'afuera', 'catedral', 'personal', 'primero', 'comisaria', 'carabineros', 'detene...",NaN
6,"Minsal detalla protocolos para uso de piscinas, cines y gimnasios durante la fase 4 del plan ""Paso a Paso""","['minsal', 'detallar', 'protocolo', 'uso', 'piscina', 'cines_gimnasios', 'paso', 'balance_diario', 'lunes', 'subsecr...",NaN
7,"""Me están extorsionando, es espantoso"": Virginia Demaría entre lágrimas revela hackeo","['extorsionar', 'espantoso', 'virginia_demario', 'lagrima', 'revelar', 'hackeo', 'figurar', 'televisivo', 'virginia_...",NaN


Cerca del 50% del dataset sin categoría (post revision de urls) quedó con un tópico al que no logramos asignarle una de nuestras categorías objetivo.

Adoptaremos dos enfoques diferentes para tratar de etiquetar correctamente estas noticias, priorizando las categorías menos numerosas con el objetivo de obtener un dataset relativamente balanceado.

### LDA al subset completo

In [41]:
df_sin_topic.to_csv('./intermediate_data/df_sin_topic.csv', index=False)

# Clasificacion de subtopicos en el cuadernillo 2. ***paso_2_re_extraccion_lda.ipynb***